#### Authentification

In [77]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='<your client id here>',
#                                                          client_secret='<your client secret here>'))

In [78]:
secrets_file = open("secrets.txt","r")

In [79]:
string = secrets_file.read()

In [80]:
#string

In [81]:
#string.split('\n')

In [82]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

#### Authentification

In [83]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [84]:
#1hMzceeWw7QiI6vaBkcEJO

# Handling large playlists - 10000 Songs

In [85]:
playlist = sp.user_playlist_tracks("spotify", "1hMzceeWw7QiI6vaBkcEJO")

In [86]:
playlist["total"] 

10000

In [87]:
len(playlist["items"])

100

Function to extract all songs from a playlist

In [88]:
playlist['next']

'https://api.spotify.com/v1/playlists/1hMzceeWw7QiI6vaBkcEJO/tracks?offset=100&limit=100&additional_types=track'

In [89]:
#sp.next(playlist)

In [90]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [ ]:
# this will take some seconds to run

all_tracks = get_playlist_tracks("1hMzceeWw7QiI6vaBkcEJO")
len(all_tracks)

# Audio features

In [62]:
# here we look only at the first page
playlist = sp.user_playlist_tracks("spotify", "1hMzceeWw7QiI6vaBkcEJO")

In [61]:
playlist["items"][12]["track"]["name"]

'Somebody to Love Me'

In [20]:
playlist["items"][78]["track"]["name"]

'Red Eyes'

In [21]:
# get the uri of a single song:
song_uri = playlist["items"][12]["track"]["uri"]
song_uri

'spotify:track:2Tky1eOpdLXo8yYawnTMcj'

In [22]:
# get the audio features for that song
sp.audio_features(song_uri)

[{'danceability': 0.694,
  'energy': 0.853,
  'key': 6,
  'loudness': -4.755,
  'mode': 0,
  'speechiness': 0.0415,
  'acousticness': 0.107,
  'instrumentalness': 1.63e-06,
  'liveness': 0.225,
  'valence': 0.566,
  'tempo': 117.896,
  'type': 'audio_features',
  'id': '2Tky1eOpdLXo8yYawnTMcj',
  'uri': 'spotify:track:2Tky1eOpdLXo8yYawnTMcj',
  'track_href': 'https://api.spotify.com/v1/tracks/2Tky1eOpdLXo8yYawnTMcj',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2Tky1eOpdLXo8yYawnTMcj',
  'duration_ms': 297933,
  'time_signature': 4}]

In [23]:
# get the uri of a single song:
song_uri = playlist["items"][78]["track"]["uri"]

In [24]:
# get the audio features for that song
sp.audio_features(song_uri)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=71jGGLe5VtEHjIk5dU2W3S:
 Max Retries, reason: too many 429 error responses

# Creating the DataFrame

In [ ]:
artist = []
uri = []
song_title= []

for i in range(len(all_tracks)):
    artist.append (all_tracks[i]["track"]["artists"][0]["name"])
    uri.append (all_tracks[i]["track"]["uri"])
    song_title.append (all_tracks[i]["track"]["name"])
    

In [ ]:
artist

In [ ]:
import pandas as pd
playlist_df = pd.DataFrame({"name":song_title, "artist":artist, "uri":uri})

In [ ]:
playlist_df

In [ ]:
audio_features = []


for i in playlist_df["uri"]:
    audio_features.append (sp.audio_features(i))

In [ ]:
audio_features

In [26]:
audio_features_df = pd.DataFrame(audio_features)

NameError: name 'pd' is not defined

In [30]:
audio_features_df

,0
0,"{'danceability': 0.823, 'energy': 0.671, 'key'..."
1,"{'danceability': 0.375, 'energy': 0.606, 'key'..."
2,"{'danceability': 0.738, 'energy': 0.491, 'key'..."
3,"{'danceability': 0.657, 'energy': 0.488, 'key'..."
4,"{'danceability': 0.753, 'energy': 0.594, 'key'..."
...,...
9995,"{'danceability': 0.824, 'energy': 0.376, 'key'..."
9996,"{'danceability': 0.673, 'energy': 0.568, 'key'..."
9997,"{'danceability': 0.684, 'energy': 0.504, 'key'..."
9998,"{'danceability': 0.677, 'energy': 0.41, 'key':..."


In [31]:
import pandas as pd
from pandas import json_normalize

audio_features_df1 = json_normalize(audio_features_df[0])

In [32]:
audio_features_df1

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.823,0.671,11,-6.437,1,0.0592,0.55100,0.001440,0.3540,0.662,125.052,audio_features,6uDlwsguxFDgbakvPyhChp,spotify:track:6uDlwsguxFDgbakvPyhChp,https://api.spotify.com/v1/tracks/6uDlwsguxFDg...,https://api.spotify.com/v1/audio-analysis/6uDl...,188697,4
1,0.375,0.606,4,-9.383,0,0.1970,0.33900,0.000924,0.4240,0.883,91.564,audio_features,00LvjMnpznr4MZFSA8x9sA,spotify:track:00LvjMnpznr4MZFSA8x9sA,https://api.spotify.com/v1/tracks/00LvjMnpznr4...,https://api.spotify.com/v1/audio-analysis/00Lv...,51373,4
2,0.738,0.491,6,-13.407,0,0.0487,0.29800,0.218000,0.1190,0.809,126.960,audio_features,3E8gEvhLia6w9lQv3hfxzM,spotify:track:3E8gEvhLia6w9lQv3hfxzM,https://api.spotify.com/v1/tracks/3E8gEvhLia6w...,https://api.spotify.com/v1/audio-analysis/3E8g...,221533,4
3,0.657,0.488,8,-10.416,0,0.0456,0.21300,0.094200,0.3600,0.662,149.763,audio_features,2jjPlj6KFzCffJ7KVQMwHX,spotify:track:2jjPlj6KFzCffJ7KVQMwHX,https://api.spotify.com/v1/tracks/2jjPlj6KFzCf...,https://api.spotify.com/v1/audio-analysis/2jjP...,216235,4
4,0.753,0.594,1,-9.486,0,0.0816,0.07110,0.000000,0.3910,0.562,112.568,audio_features,3seHx466iTcUmWE9dOVB3v,spotify:track:3seHx466iTcUmWE9dOVB3v,https://api.spotify.com/v1/tracks/3seHx466iTcU...,https://api.spotify.com/v1/audio-analysis/3seH...,193093,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.824,0.376,3,-11.921,1,0.2200,0.70500,0.011500,0.1060,0.819,94.013,audio_features,57BxU4uqdbl6TITWtnQDMw,spotify:track:57BxU4uqdbl6TITWtnQDMw,https://api.spotify.com/v1/tracks/57BxU4uqdbl6...,https://api.spotify.com/v1/audio-analysis/57Bx...,324608,4
9996,0.673,0.568,9,-6.708,0,0.2390,0.50100,0.000881,0.1390,0.568,170.131,audio_features,7lL6ND4Jj1tWpoDFaJoxtS,spotify:track:7lL6ND4Jj1tWpoDFaJoxtS,https://api.spotify.com/v1/tracks/7lL6ND4Jj1tW...,https://api.spotify.com/v1/audio-analysis/7lL6...,143533,4
9997,0.684,0.504,9,-8.913,1,0.0801,0.09720,0.000290,0.3360,0.740,86.226,audio_features,1PNaGC2ihDmVCldSce119E,spotify:track:1PNaGC2ihDmVCldSce119E,https://api.spotify.com/v1/tracks/1PNaGC2ihDmV...,https://api.spotify.com/v1/audio-analysis/1PNa...,216601,4
9998,0.677,0.410,9,-10.598,0,0.0400,0.59100,0.103000,0.0813,0.515,81.924,audio_features,2887tlqmjY4WcyQ3Dy6dZy,spotify:track:2887tlqmjY4WcyQ3Dy6dZy,https://api.spotify.com/v1/tracks/2887tlqmjY4W...,https://api.spotify.com/v1/audio-analysis/2887...,274013,4


In [33]:
df_final = pd.concat([playlist_df, audio_features_df1], axis=1)

In [34]:
df_final

,name,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,This Magic Moment - Crisp.nl Version,MassiveMusic,spotify:track:6uDlwsguxFDgbakvPyhChp,0.823,0.671,11,-6.437,1,0.0592,0.55100,...,0.3540,0.662,125.052,audio_features,6uDlwsguxFDgbakvPyhChp,spotify:track:6uDlwsguxFDgbakvPyhChp,https://api.spotify.com/v1/tracks/6uDlwsguxFDg...,https://api.spotify.com/v1/audio-analysis/6uDl...,188697,4
1,00h00,Hoosky,spotify:track:00LvjMnpznr4MZFSA8x9sA,0.375,0.606,4,-9.383,0,0.1970,0.33900,...,0.4240,0.883,91.564,audio_features,00LvjMnpznr4MZFSA8x9sA,spotify:track:00LvjMnpznr4MZFSA8x9sA,https://api.spotify.com/v1/tracks/00LvjMnpznr4...,https://api.spotify.com/v1/audio-analysis/00Lv...,51373,4
2,Under My Thumb,The Rolling Stones,spotify:track:3E8gEvhLia6w9lQv3hfxzM,0.738,0.491,6,-13.407,0,0.0487,0.29800,...,0.1190,0.809,126.960,audio_features,3E8gEvhLia6w9lQv3hfxzM,spotify:track:3E8gEvhLia6w9lQv3hfxzM,https://api.spotify.com/v1/tracks/3E8gEvhLia6w...,https://api.spotify.com/v1/audio-analysis/3E8g...,221533,4
3,Conquer Me,Jane B,spotify:track:2jjPlj6KFzCffJ7KVQMwHX,0.657,0.488,8,-10.416,0,0.0456,0.21300,...,0.3600,0.662,149.763,audio_features,2jjPlj6KFzCffJ7KVQMwHX,spotify:track:2jjPlj6KFzCffJ7KVQMwHX,https://api.spotify.com/v1/tracks/2jjPlj6KFzCf...,https://api.spotify.com/v1/audio-analysis/2jjP...,216235,4
4,Dapper (feat. Anderson .Paak),Domo Genesis,spotify:track:3seHx466iTcUmWE9dOVB3v,0.753,0.594,1,-9.486,0,0.0816,0.07110,...,0.3910,0.562,112.568,audio_features,3seHx466iTcUmWE9dOVB3v,spotify:track:3seHx466iTcUmWE9dOVB3v,https://api.spotify.com/v1/tracks/3seHx466iTcU...,https://api.spotify.com/v1/audio-analysis/3seH...,193093,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Lijay,Mulatu Astatke,spotify:track:57BxU4uqdbl6TITWtnQDMw,0.824,0.376,3,-11.921,1,0.2200,0.70500,...,0.1060,0.819,94.013,audio_features,57BxU4uqdbl6TITWtnQDMw,spotify:track:57BxU4uqdbl6TITWtnQDMw,https://api.spotify.com/v1/tracks/57BxU4uqdbl6...,https://api.spotify.com/v1/audio-analysis/57Bx...,324608,4
9996,Pretty Nice Life (feat. Tiken Jah Fakoly),M.I.L.K.,spotify:track:7lL6ND4Jj1tWpoDFaJoxtS,0.673,0.568,9,-6.708,0,0.2390,0.50100,...,0.1390,0.568,170.131,audio_features,7lL6ND4Jj1tWpoDFaJoxtS,spotify:track:7lL6ND4Jj1tWpoDFaJoxtS,https://api.spotify.com/v1/tracks/7lL6ND4Jj1tW...,https://api.spotify.com/v1/audio-analysis/7lL6...,143533,4
9997,Take It Slow - Mousse T. Remix,Boozoo Bajou,spotify:track:1PNaGC2ihDmVCldSce119E,0.684,0.504,9,-8.913,1,0.0801,0.09720,...,0.3360,0.740,86.226,audio_features,1PNaGC2ihDmVCldSce119E,spotify:track:1PNaGC2ihDmVCldSce119E,https://api.spotify.com/v1/tracks/1PNaGC2ihDmV...,https://api.spotify.com/v1/audio-analysis/1PNa...,216601,4
9998,Birds of a Feather,Mocky,spotify:track:2887tlqmjY4WcyQ3Dy6dZy,0.677,0.410,9,-10.598,0,0.0400,0.59100,...,0.0813,0.515,81.924,audio_features,2887tlqmjY4WcyQ3Dy6dZy,spotify:track:2887tlqmjY4WcyQ3Dy6dZy,https://api.spotify.com/v1/tracks/2887tlqmjY4W...,https://api.spotify.com/v1/audio-analysis/2887...,274013,4


In [1]:
df_final.to_csv('my_playlist.csv', index=False)

NameError: name 'df_final' is not defined